<a href="https://colab.research.google.com/github/VladousSparrowous/ast-finetune/blob/main/hse_detect_ast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hse-itmo-acoustic-event-detection-2025/sample_submission.csv
/kaggle/input/hse-itmo-acoustic-event-detection-2025/train.csv
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/88e609968f5211ded8a3.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/c9c303f8bd1fe11bd48b.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/66912b0b6b053d8ded03.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/706e8c33fcdf1b348247.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/a4c7f24f6e5e876fbb4f.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/7ec836172449bcdcb855.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/c6fc30d56b819c3dd217.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/6d5a3fa35b67ba8f0a31.wav
/kaggle/input/hse-itmo-acoustic-event-detection-2025/audio_train/train/8960424988a11694d8b6.wa

In [ ]:
# set seeds
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
# path to your train/test/meta folders
DATA_PATH = '/kaggle/input/hse-itmo-acoustic-event-detection-2025/'

# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = 'audio_train/train'
test_data_folder = 'audio_test/test'

In [ ]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [ ]:
n_classes = df_train.label.nunique()
print(n_classes)
#classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}
#df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak
2,17bb93b73b8e79234cb3.wav,Electric_piano
3,7d5c7a40a936136da55e.wav,Harmonica
4,17e0ee7565a33d6c2326.wav,Snare_drum


In [ ]:
import pandas as pd
import librosa
from datasets import Dataset

class AudioDataset:
    def __init__(self, df):
        self.df = df
        self.labels = df['label'].tolist()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        fname = os.path.join(DATA_PATH, train_data_folder, self.df.iloc[idx]['fname'])
        label = self.df.iloc[idx]['label']

        audio, sr = librosa.load(fname, sr=16000)

        return {
            "audio": {'path': fname, 'array': audio, 'sampling_rate': sr},
            "labels": label
        }

audio_dataset = AudioDataset(df_train)
dataset = Dataset.from_list([audio_dataset[i] for i in range(len(audio_dataset))])

In [ ]:
print(dataset[0])

{'audio': {'array': [0.000451431842520833, 0.00020403554663062096, -2.4508815840817988e-05, 0.00023131753550842404, -4.4411266571842134e-05, 0.00013861230399925262, -0.00019832242105621845, -7.890642882557586e-05, 0.0002866786962840706, -0.00018064098549075425, 0.0004987957654520869, 1.051361959980568e-05, -4.5632245019078255e-05, 0.0002461757103446871, 0.0007338072755374014, 0.0015417217509821057, 0.0011569736525416374, 0.0010786682832986116, -5.613726534647867e-05, -3.2582684070803225e-05, 0.0003969328827224672, 0.001058411318808794, 0.0011989740887656808, 1.9304370653117076e-05, -0.0012319819070398808, -0.001543097198009491, -0.00023921308456920087, 0.00021071948867756873, 0.0003059454611502588, 2.498111643944867e-05, -0.00013971605221740901, -0.00017958749958779663, -0.0005715606384910643, 0.0001536837953608483, 0.000303277833154425, 0.000857472070492804, 0.0006782257696613669, 5.3708685300080106e-05, -0.0011427997378632426, -0.001263726968318224, 0.0002471182669978589, -6.36969925

In [ ]:
from transformers import ASTFeatureExtractor

pretrained_model = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(pretrained_model)

model_input_name = feature_extractor.model_input_names[0]
SAMPLING_RATE = feature_extractor.sampling_rate

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
from datasets import ClassLabel
class_names = sorted(df_train['label'].unique())
dataset = dataset.cast_column('labels', ClassLabel(names=class_names))

Casting the dataset:   0%|          | 0/5683 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=0, stratify_by_column="labels")

In [ ]:
def preprocess_audio(batch):
    wavs = [audio["array"] for audio in batch["input_values"]]
    inputs = feature_extractor(wavs, sampling_rate=SAMPLING_RATE, return_tensors="pt")

    output_batch = {model_input_name: inputs.get(model_input_name), "labels": list(batch["labels"])}
    return output_batch


dataset = dataset.rename_column("audio", "input_values")
dataset.set_transform(preprocess_audio, output_all_columns=False)

In [ ]:
from tqdm import tqdm
feature_extractor.do_normalize = False
std = []
mean = []

dataset["train"].set_transform(preprocess_audio, output_all_columns=False)
for i, (audio_input, labels) in tqdm(enumerate(dataset["train"])):
    cur_mean = torch.mean(dataset["train"][i][audio_input])
    cur_std = torch.std(dataset["train"][i][audio_input])
    mean.append(cur_mean)
    std.append(cur_std)

feature_extractor.mean = np.mean(mean)
feature_extractor.std = np.mean(std)
feature_extractor.do_normalize = True

4546it [11:44,  6.45it/s]


In [ ]:
from transformers import ASTConfig, ASTForAudioClassification


config = ASTConfig.from_pretrained(pretrained_model)

label2id = {cl:i for i,cl in enumerate(df_train.label.unique())}
config.num_labels = n_classes
config.label2id = label2id
config.id2label = {v: k for k, v in label2id.items()}


model = ASTForAudioClassification.from_pretrained(pretrained_model, config=config, ignore_mismatched_sizes=True)
model.init_weights()

2025-11-25 21:20:54.803735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764105655.041522      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764105655.107977      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([41]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([41, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./runs/ast_classifier",
    logging_dir="./logs/ast_classifier",
    report_to="tensorboard",
    learning_rate=3e-5,
    push_to_hub=False,
    num_train_epochs=15,
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_strategy="steps",
    logging_steps=100,
)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [ ]:
import evaluate
import numpy as np

f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    predictions = np.argmax(logits, axis=1)
    metrics = f1.compute(predictions=predictions, references=eval_pred.label_ids, average='macro')
    return metrics

In [ ]:
from transformers import Trainer

# Setup the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.054400,1.041705,0.839840
2,0.064500,1.109414,0.845975
3,0.051500,1.223216,0.858818
4,0.017300,1.440284,0.855740
5,0.026000,1.576989,0.862679
6,0.000000,1.543359,0.867807
7,0.000300,1.438313,0.869867
8,0.000100,1.371059,0.875716
9,0.000000,1.395326,0.873793
10,0.000000,1.383963,0.875995


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=4275, training_loss=0.01237110402658582, metrics={'train_runtime': 15066.3293, 'train_samples_per_second': 4.526, 'train_steps_per_second': 0.284, 'total_flos': 4.623720173674168e+18, 'train_loss': 0.01237110402658582, 'epoch': 15.0})

In [ ]:

if hasattr(feature_extractor, 'mean'):
    feature_extractor.mean = float(feature_extractor.mean)
if hasattr(feature_extractor, 'std'):
    feature_extractor.std = float(feature_extractor.std)

trainer.save_model("./final-ast-model")
feature_extractor.save_pretrained("./final-ast-model")

['./final-ast-model/preprocessor_config.json']

In [ ]:
from transformers import ASTFeatureExtractor, ASTForAudioClassification

model_path = "/kaggle/input/ast-hse/other/default/1/final-ast-model"
model = ASTForAudioClassification.from_pretrained(model_path, local_files_only=True)
feature_extractor = ASTFeatureExtractor.from_pretrained(model_path, local_files_only=True)
model_input_name = feature_extractor.model_input_names[0]
feature_extractor.do_normalize = True
model.eval()


DATA_PATH = '/kaggle/input/hse-itmo-acoustic-event-detection-2025/'
test_data_folder = 'audio_test/test'
test_folder_path = os.path.join(DATA_PATH, test_data_folder)

In [ ]:
test_meta_fname = 'sample_submission.csv'
df_sub = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_sub_answ = df_sub.copy()

In [ ]:
for index, rows in tqdm(df_sub.iterrows()):
    file_path = os.path.join(test_folder_path, rows['fname'])
    predicted_label = predict_single_audio(file_path)
    df_sub_answ.loc[index, 'label'] = predicted_label
df_sub_answ.to_csv("submission.csv", index=False)


In [ ]:
import os
import torch
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

def predict_single_audio(audio_path):

    audio_array, sampling_rate = librosa.load(audio_path, sr=16000)

    inputs = feature_extractor(
        audio_array,
        sampling_rate=sampling_rate,
        return_tensors="pt",
        padding=True
    )

    with torch.no_grad():

        outputs = model(**inputs)
        predictions = int(torch.argmax(outputs.logits, axis=1)[0])
    predicted_label = model.config.id2label[predictions]
    return predicted_label


In [ ]:
#тут приходьтся поменять метки классов так как я их напутал вначале

data_label = list(enumerate(class_names))
data_label_dict = {}
for i in data_label:
    data_label_dict[i[0]] = i[1]

model_label = model.config.label2id

for j in model_label:
    model_label[j] = data_label_dict[model_label[j]]

for index, rows in tqdm(df_sub_answ.iterrows()):
    df_sub_answ.loc[index, 'label'] = model_label[df_sub_answ.loc[index, 'label']]

df_sub_answ.to_csv("submission.csv", index=False)